In [1]:
import time

import gym
import gym_snake
from ml_tools import q_learning

import numpy as np
import matplotlib.pyplot as plt

from ml_tools.ui import UI

ui = UI()
ui.connect("./public")

'127.0.0.1:54161'

In [2]:
model_name = "01_model_game-trained"

# brain_name = "baseline"
# brain_name = "pre-trained/baseline"
brain_name = "pre-trained/all-trainable"

# brain_name = "fast-epsilon-decay/pre-trained/all-trainable"

load_pre_trained = True

In [3]:
from brain.v2.brain import Brain
from preprocessor.v1.preprocessor import Preprocessor

preprocessor = Preprocessor()
preprocessor.ui = ui
brain = Brain(learning_rate=0)
brain.ui = ui
brain.load_model(name=brain_name, model_name=model_name)

brain.model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 50, 50, 3)]       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 11, 11, 32)        6176      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 6, 6, 32)         0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 2, 2, 64)          32832     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 2, 2, 64)         0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 256)               0   

In [4]:
agent = q_learning.Agent(
  action_space=[0, 1, 2, 3],

  brain=brain, preprocessor=preprocessor,

  epsilon_start = 0,
  epsilon_stop = 0,
  epsilon_decay_num_steps = 250,
  
  discount_factor=0.97,
  
  replay_batch_size=32*2,
  model_batch_size=16,

  steps_to_train=20,
  steps_to_update_target=40,

  replay_memory_max_len=50_000,
  replay_memory_min_len= 2_000,
)
agent.training = False

env = gym.make("snake-v0")

INFO:tensorflow:Assets written to: /var/folders/f1/dwpfqnws1qs_gmw3dgwzykyc0000gn/T/tmp7hsghhtqml_tools-84208-AsyncFitModel/assets


INFO:tensorflow:Assets written to: /var/folders/f1/dwpfqnws1qs_gmw3dgwzykyc0000gn/T/tmp7hsghhtqml_tools-84208-AsyncFitModel/assets


In [5]:
raw_observation = env.reset()
agent.reset(raw_observation)

while True:
  ui.video.update_stream("raw", raw_observation)

  speed = int(ui.control.get_value("speed") or 100)
  if speed != 100:
    time.sleep(1 / speed)

  agent.training = True if (ui.control.get_value("training") or "true") == "true" else False
  action = agent.act()
  raw_next_observation, reward, done, info = env.step(action)
  agent.step(reward, raw_next_observation)

  raw_observation = raw_next_observation
  if done:
    raw_observation = env.reset()
    agent.reset(raw_observation)

AttributeError: module 'numpy' has no attribute 'int'

Process Process-1:
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/aaa/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/opt/homebrew/Caskroom/miniforge/base/envs/aaa/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/wiktorjurkiewicz/repos/00_PROJECTS/@_GENERAL/04_CODE/ML/snake-deep-q-learning/ml_tools/tensorflow/keras/async_model.py", line 55, in _fit_process_start
    (args, kwargs) = fit_queue.get()
  File "/opt/homebrew/Caskroom/miniforge/base/envs/aaa/lib/python3.9/multiprocessing/queues.py", line 103, in get
    res = self._recv_bytes()
  File "/opt/homebrew/Caskroom/miniforge/base/envs/aaa/lib/python3.9/multiprocessing/connection.py", line 221, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/opt/homebrew/Caskroom/miniforge/base/envs/aaa/lib/python3.9/multiprocessing/connection.py", line 419, in _recv_bytes
    buf = se